In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import time
import os

# Current dir
current_dir = os.getcwd()

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),  # Instalar el controlador automáticamente
    options=options
)
url ='https://apps.bea.gov/iTable/?reqid=70&step=1&isuri=1&acrdn=6#eyJhcHBpZCI6NzAsInN0ZXBzIjpbMSwyOSwyNSwzMSwyNiwyNywzMF0sImRhdGEiOltbIlRhYmxlSWQiLCIzNiJdLFsiTWFqb3JfQXJlYSIsIjAiXSxbIlN0YXRlIixbIjAiXV0sWyJBcmVhIixbIlhYIl1dLFsiU3RhdGlzdGljIixbIjMiXV0sWyJVbml0X29mX21lYXN1cmUiLCJMZXZlbHMiXSxbIlllYXIiLFsiLTEiXV0sWyJZZWFyQmVnaW4iLCItMSJdLFsiWWVhcl9FbmQiLCItMSJdXX0='
driver.get(url)

### Riqueza per cápita (web scrapping)

In [28]:
# Configuramos un objeto de espera con un tiempo máximo de 30 segundos.
# Esto nos permite esperar hasta que ciertas condiciones se cumplan en la página.
wait = WebDriverWait(driver, 30)

# Esperamos hasta que la tabla con ID 'DataTbl' sea visible en la página.
# Esto asegura que la tabla dinámica esté completamente renderizada antes de continuar.
table = wait.until(EC.visibility_of_element_located((By.ID, 'DataTbl')))

# Esperamos hasta que la instancia de DataTables para 'DataTbl' esté inicializada.
# Usamos JavaScript para verificar si DataTables está listo, asegurando que todo el contenido dinámico esté cargado.
wait.until(lambda driver: driver.execute_script("return typeof $('#DataTbl').DataTable() !== 'undefined';"))

# Usamos la API de DataTables para obtener todos los encabezados de las columnas.
# Ejecutamos JavaScript que accede a la tabla, obtiene los encabezados de las columnas y los convierte en una lista.
column_headers = driver.execute_script("""
var table = $('#DataTbl').DataTable();
return table.columns().header().map(function(th) {
    return $(th).text().trim();
}).toArray();
""")

# Buscamos el elemento <tbody> dentro de la tabla, que contiene las filas de datos.
tbody = table.find_element(By.TAG_NAME, 'tbody')

# Buscamos todos los elementos <tr> dentro del <tbody>, que representan las filas de la tabla.
rows = tbody.find_elements(By.TAG_NAME, 'tr')

# Inicializamos una lista vacía para almacenar los datos extraídos de las filas.
data = []

# Iteramos sobre cada fila de la tabla.
for row in rows:
    # Dentro de cada fila, buscamos todos los elementos <td>, que son las celdas de datos.
    cells = row.find_elements(By.TAG_NAME, 'td')
    # Extraemos el texto de cada celda, eliminando espacios en blanco adicionales, y lo añadimos a la lista 'data'.
    data.append([cell.text.strip() for cell in cells])

# Creamos un DataFrame de pandas usando los datos extraídos y los encabezados de las columnas obtenidos anteriormente.
capita_personal_income = pd.DataFrame(data, columns=column_headers)

# Imprimimos las primeras filas del DataFrame para verificar que los datos se extrajeron correctamente.
print(capita_personal_income.head())

# Cerramos la sesión del navegador para liberar recursos.
driver.quit()

  GeoFips        GeoName 1948:Q1 1948:Q2 1948:Q3 1948:Q4 1949:Q1 1949:Q2  \
0   00000  United States    (NA)    (NA)    (NA)    (NA)    (NA)    (NA)   
1   01000        Alabama    (NA)    (NA)    (NA)    (NA)    (NA)    (NA)   
2   02000       Alaska *    (NA)    (NA)    (NA)    (NA)    (NA)    (NA)   
3   04000        Arizona    (NA)    (NA)    (NA)    (NA)    (NA)    (NA)   
4   05000       Arkansas    (NA)    (NA)    (NA)    (NA)    (NA)    (NA)   

  1949:Q3 1949:Q4  ... 2022:Q3 2022:Q4 2023:Q1 2023:Q2 2023:Q3 2023:Q4  \
0    (NA)    (NA)  ...  66,644  67,527  68,383  69,157  69,709  70,340   
1    (NA)    (NA)  ...  52,080  52,798  53,411  53,847  54,308  54,869   
2    (NA)    (NA)  ...  69,367  70,276  69,651  70,925  71,981  72,593   
3    (NA)    (NA)  ...  59,350  60,255  61,276  62,190  62,408  62,804   
4    (NA)    (NA)  ...  55,969  57,031  57,509  57,487  57,548  57,880   

  2024:Q1 2024:Q2 2024:Q3 2024:Q4  
0  71,763  72,268  72,537  73,251  
1  55,930  56,395  56,893 

- Verificamos si están todos los encabezados en las columnas

In [29]:
capita_personal_income.columns

Index(['GeoFips', 'GeoName', '1948:Q1', '1948:Q2', '1948:Q3', '1948:Q4',
       '1949:Q1', '1949:Q2', '1949:Q3', '1949:Q4',
       ...
       '2022:Q3', '2022:Q4', '2023:Q1', '2023:Q2', '2023:Q3', '2023:Q4',
       '2024:Q1', '2024:Q2', '2024:Q3', '2024:Q4'],
      dtype='object', length=310)

In [30]:
capita_personal_income.head()

,GeoFips,GeoName,1948:Q1,1948:Q2,1948:Q3,1948:Q4,1949:Q1,1949:Q2,1949:Q3,1949:Q4,...,2022:Q3,2022:Q4,2023:Q1,2023:Q2,2023:Q3,2023:Q4,2024:Q1,2024:Q2,2024:Q3,2024:Q4
0,00000,United States,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,"66,644","67,527","68,383","69,157","69,709","70,340","71,763","72,268","72,537","73,251"
1,01000,Alabama,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,"52,080","52,798","53,411","53,847","54,308","54,869","55,930","56,395","56,893","57,551"
2,02000,Alaska *,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,"69,367","70,276","69,651","70,925","71,981","72,593","74,604","74,705","75,469","76,299"
3,04000,Arizona,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,"59,350","60,255","61,276","62,190","62,408","62,804","64,225","64,432","64,364","64,904"
4,05000,Arkansas,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,"55,969","57,031","57,509","57,487","57,548","57,880","58,869","59,158","60,090","60,549"


- Extraer solo los años que necesitamos

In [31]:
# Extrae los primeros cuatro caracteres de cada nombre de columna
first_four = capita_personal_income.columns.str[:4]

# Convierte a números, convirtiendo valores no numéricos a NaN
years = pd.to_numeric(first_four, errors='coerce')

# Crea una máscara para columnas donde el año está entre 2011 y 2016
mask = ~years.isna() & (years >= 2011) & (years <= 2016)

# Selecciona 'GeoName' y las columnas de 2011-2016
columns_to_keep = ['GeoName'] + capita_personal_income.columns[mask].tolist()

capita_personal_income = capita_personal_income[columns_to_keep]

capita_personal_income

,GeoName,2011:Q1,2011:Q2,2011:Q3,2011:Q4,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,...,2014:Q3,2014:Q4,2015:Q1,2015:Q2,2015:Q3,2015:Q4,2016:Q1,2016:Q2,2016:Q3,2016:Q4
0,United States,"42,230","42,501","42,865","42,974","43,757","44,077","43,837","45,249","43,992",...,"46,560","47,103","47,602","47,976","48,238","48,402","48,552","48,713","49,090","49,528"
1,Alabama,"34,726","34,753","34,977","35,050","35,379","35,611","35,191","36,030","35,586",...,"36,954","37,418","37,799","38,115","38,415","38,397","38,485","38,613","38,714","38,983"
2,Alaska *,"51,311","52,099","52,784","53,434","53,209","53,429","53,064","53,682","52,153",...,"55,315","55,905","56,615","56,770","56,699","56,642","55,434","55,408","55,570","55,928"
3,Arizona,"34,975","35,111","35,345","35,548","36,021","36,298","35,978","37,134","36,518",...,"38,394","38,878","39,480","39,848","39,884","40,269","40,598","41,011","41,424","41,646"
4,Arkansas,"33,850","33,838","34,346","34,712","35,788","36,251","35,854","37,258","36,159",...,"38,371","39,015","38,777","39,068","39,471","39,722","39,817","40,075","40,228","40,580"
5,California,"45,195","45,315","45,768","45,726","46,904","47,372","47,201","49,642","47,185",...,"51,089","51,735","52,906","53,680","54,124","54,562","55,214","55,473","56,109","56,653"
6,Colorado,"43,230","43,588","44,063","44,048","44,926","45,242","44,893","46,854","46,280",...,"50,474","51,187","51,282","51,554","51,406","51,393","50,857","50,964","51,438","52,031"
7,Connecticut,"62,209","63,162","63,434","63,042","63,054","63,547","62,730","64,896","61,739",...,"64,812","65,753","65,553","66,018","66,329","66,960","67,327","67,493","67,551","67,754"
8,Delaware,"43,339","43,684","44,003","44,059","43,533","43,818","43,254","44,423","43,347",...,"45,629","45,825","46,764","47,379","47,224","47,573","47,474","48,000","48,385","48,846"
9,District of Columbia,"66,550","66,808","67,321","67,185","67,609","67,644","66,580","68,176","66,992",...,"70,974","71,274","74,281","74,863","75,010","75,095","76,752","77,358","77,615","78,293"


- Convertir los numeros a float

In [32]:
# Identifica todas las columnas salvo GeoName
num_cols = [c for c in capita_personal_income.columns if c != 'GeoName']

# Elimina las comas y convierte a float
capita_personal_income[num_cols] = (
    capita_personal_income[num_cols]
      .replace(',', '', regex=True)   # quita todas las comas
      .astype(float)                  # convierte a float
)

# Comprueba tipos
print(capita_personal_income.dtypes)


GeoName     object
2011:Q1    float64
2011:Q2    float64
2011:Q3    float64
2011:Q4    float64
2012:Q1    float64
2012:Q2    float64
2012:Q3    float64
2012:Q4    float64
2013:Q1    float64
2013:Q2    float64
2013:Q3    float64
2013:Q4    float64
2014:Q1    float64
2014:Q2    float64
2014:Q3    float64
2014:Q4    float64
2015:Q1    float64
2015:Q2    float64
2015:Q3    float64
2015:Q4    float64
2016:Q1    float64
2016:Q2    float64
2016:Q3    float64
2016:Q4    float64
dtype: object


- Pasar Q1, Q2, Q3 y Q4 a sus meses correspondientes

In [33]:
import re

# 1) Identificar columnas trimestrales
quarterly_cols = [
    col for col in capita_personal_income.columns
    if re.match(r'^\d{4}:Q[1-4]$', col)
]

# 2) Cambiar a formato largo
df_long = pd.melt(
    capita_personal_income,
    id_vars=['GeoName'],
    value_vars=quarterly_cols,
    var_name='Quarter',
    value_name='Quarterly_Income'
)

# 3) Convertir ingresos a numérico (quitando comas) y calcular ingreso mensual
df_long['Quarterly_Income'] = (
    df_long['Quarterly_Income']
      .astype(str)
      .str.replace(',', '', regex=False)
      .pipe(pd.to_numeric, errors='coerce')
)
df_long['Monthly_Income'] = df_long['Quarterly_Income'] / 3

# 4) Extraer año y trimestre
df_long[['Year','Q']] = df_long['Quarter'].str.split(':Q', expand=True)
df_long['Year'] = df_long['Year'].astype(int)
df_long['Q'] = 'Q' + df_long['Q']

# 5) Mapear cada trimestre a sus meses en español
months_map = {
    'Q1': ['enero', 'febrero', 'marzo'],
    'Q2': ['abril', 'mayo', 'junio'],
    'Q3': ['julio', 'agosto', 'septiembre'],
    'Q4': ['octubre', 'noviembre', 'diciembre']
}

# 6) Crear columna con lista de meses_año
df_long['Month'] = df_long.apply(
    lambda row: [f"{m}_{row['Year']}" for m in months_map[row['Q']]],
    axis=1
)

# 7) Explode para tener una fila por cada mes
df_monthly = df_long.explode('Month')[['GeoName','Month','Monthly_Income']]

# 8) Pivotar a formato ancho
capita_personal_income_final = (
    df_monthly
      .pivot(index='GeoName', columns='Month', values='Monthly_Income')
      .reset_index()
)

# 9) Ordenar columnas cronológicamente usando nombres de meses en español
months = [
    'enero','febrero','marzo','abril','mayo','junio',
    'julio','agosto','septiembre','octubre','noviembre','diciembre'
]
month_order = {m: i+1 for i, m in enumerate(months)}

other_cols = [c for c in capita_personal_income_final.columns if c != 'GeoName']
ordered = sorted(
    other_cols,
    key=lambda col: (
        int(col.split('_')[1]),           # año
        month_order[col.split('_')[0]]     # posición del mes
    )
)
capita_personal_income_final = capita_personal_income_final[['GeoName'] + ordered]


# 10) Mostrar

capita_personal_income_final


Month,GeoName,enero_2011,febrero_2011,marzo_2011,abril_2011,mayo_2011,junio_2011,julio_2011,agosto_2011,septiembre_2011,...,marzo_2016,abril_2016,mayo_2016,junio_2016,julio_2016,agosto_2016,septiembre_2016,octubre_2016,noviembre_2016,diciembre_2016
0,Alabama,11575.333333,11575.333333,11575.333333,11584.333333,11584.333333,11584.333333,11659.000000,11659.000000,11659.000000,...,12828.333333,12871.000000,12871.000000,12871.000000,12904.666667,12904.666667,12904.666667,12994.333333,12994.333333,12994.333333
1,Alaska *,17103.666667,17103.666667,17103.666667,17366.333333,17366.333333,17366.333333,17594.666667,17594.666667,17594.666667,...,18478.000000,18469.333333,18469.333333,18469.333333,18523.333333,18523.333333,18523.333333,18642.666667,18642.666667,18642.666667
2,Arizona,11658.333333,11658.333333,11658.333333,11703.666667,11703.666667,11703.666667,11781.666667,11781.666667,11781.666667,...,13532.666667,13670.333333,13670.333333,13670.333333,13808.000000,13808.000000,13808.000000,13882.000000,13882.000000,13882.000000
3,Arkansas,11283.333333,11283.333333,11283.333333,11279.333333,11279.333333,11279.333333,11448.666667,11448.666667,11448.666667,...,13272.333333,13358.333333,13358.333333,13358.333333,13409.333333,13409.333333,13409.333333,13526.666667,13526.666667,13526.666667
4,California,15065.000000,15065.000000,15065.000000,15105.000000,15105.000000,15105.000000,15256.000000,15256.000000,15256.000000,...,18404.666667,18491.000000,18491.000000,18491.000000,18703.000000,18703.000000,18703.000000,18884.333333,18884.333333,18884.333333
5,Colorado,14410.000000,14410.000000,14410.000000,14529.333333,14529.333333,14529.333333,14687.666667,14687.666667,14687.666667,...,16952.333333,16988.000000,16988.000000,16988.000000,17146.000000,17146.000000,17146.000000,17343.666667,17343.666667,17343.666667
6,Connecticut,20736.333333,20736.333333,20736.333333,21054.000000,21054.000000,21054.000000,21144.666667,21144.666667,21144.666667,...,22442.333333,22497.666667,22497.666667,22497.666667,22517.000000,22517.000000,22517.000000,22584.666667,22584.666667,22584.666667
7,Delaware,14446.333333,14446.333333,14446.333333,14561.333333,14561.333333,14561.333333,14667.666667,14667.666667,14667.666667,...,15824.666667,16000.000000,16000.000000,16000.000000,16128.333333,16128.333333,16128.333333,16282.000000,16282.000000,16282.000000
8,District of Columbia,22183.333333,22183.333333,22183.333333,22269.333333,22269.333333,22269.333333,22440.333333,22440.333333,22440.333333,...,25584.000000,25786.000000,25786.000000,25786.000000,25871.666667,25871.666667,25871.666667,26097.666667,26097.666667,26097.666667
9,Far West *,14706.000000,14706.000000,14706.000000,14752.000000,14752.000000,14752.000000,14880.000000,14880.000000,14880.000000,...,17839.000000,17919.333333,17919.333333,17919.333333,18113.666667,18113.666667,18113.666667,18286.666667,18286.666667,18286.666667


- Eliminar arteriscos de GeoName

In [35]:
capita_personal_income_final['GeoName'] = (
    capita_personal_income_final['GeoName']
      .str.replace(r'\s*\*\s*', '', regex=True)
      .str.strip()
)

capita_personal_income_final.head()

Month,GeoName,enero_2011,febrero_2011,marzo_2011,abril_2011,mayo_2011,junio_2011,julio_2011,agosto_2011,septiembre_2011,...,marzo_2016,abril_2016,mayo_2016,junio_2016,julio_2016,agosto_2016,septiembre_2016,octubre_2016,noviembre_2016,diciembre_2016
0,Alabama,11575.333333,11575.333333,11575.333333,11584.333333,11584.333333,11584.333333,11659.000000,11659.000000,11659.000000,...,12828.333333,12871.000000,12871.000000,12871.000000,12904.666667,12904.666667,12904.666667,12994.333333,12994.333333,12994.333333
1,Alaska,17103.666667,17103.666667,17103.666667,17366.333333,17366.333333,17366.333333,17594.666667,17594.666667,17594.666667,...,18478.000000,18469.333333,18469.333333,18469.333333,18523.333333,18523.333333,18523.333333,18642.666667,18642.666667,18642.666667
2,Arizona,11658.333333,11658.333333,11658.333333,11703.666667,11703.666667,11703.666667,11781.666667,11781.666667,11781.666667,...,13532.666667,13670.333333,13670.333333,13670.333333,13808.000000,13808.000000,13808.000000,13882.000000,13882.000000,13882.000000
3,Arkansas,11283.333333,11283.333333,11283.333333,11279.333333,11279.333333,11279.333333,11448.666667,11448.666667,11448.666667,...,13272.333333,13358.333333,13358.333333,13358.333333,13409.333333,13409.333333,13409.333333,13526.666667,13526.666667,13526.666667
4,California,15065.000000,15065.000000,15065.000000,15105.000000,15105.000000,15105.000000,15256.000000,15256.000000,15256.000000,...,18404.666667,18491.000000,18491.000000,18491.000000,18703.000000,18703.000000,18703.000000,18884.333333,18884.333333,18884.333333


capita_personal_income_final['GeoName'] = (
    capita_personal_income_final['GeoName']
      .str.replace(r'\s*\*\s*', '', regex=True)
      .str.strip()
)

In [36]:
# Directorio “data” junto al notebook
data_dir = os.path.join(current_dir, 'data')
os.makedirs(data_dir, exist_ok=True)

# Guardar allí el CSV
capital_personal_income_file = os.path.join(data_dir, "riqueza_per_capita.csv")
capita_personal_income_final.to_csv(capital_personal_income_file, index=False)
print(f"Guardado en: {capital_personal_income_file}")


Guardado en: c:\Users\jcuenca\Downloads\data\riqueza_per_capita.csv
